In [49]:
import uuid
import random
import numpy as np

In [50]:
from tax_calculation import get_taxable_income, calculate_final_tax

In [51]:
from sqlalchemy import create_engine, Column, String, Integer, DateTime
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

In [52]:
DB_URL = (
    "postgresql://test:must_be_eight_characters"
    + "@example-1.cluster-cculi2axzscc.us-east-1.rds.amazonaws.com:5432/test"
)

engine = create_engine(DB_URL)

# Create a session to interact with the database
Session = sessionmaker(bind=engine)
session = Session()

In [53]:
# Create a base class for declarative models
Base = declarative_base()


# Define the User model
class User(Base):
    __tablename__ = "users"

    id = Column(String, primary_key=True)
    name = Column(String)
    password = Column(String)
    gender = Column(String)
    date_of_birth = Column(DateTime)


# Define the TaxDetails model
class TaxDetails(Base):
    __tablename__ = "tax_details"

    tax_id = Column(String, primary_key=True)
    user_id = Column(String)
    year = Column(Integer)
    income = Column(Integer)
    taxable_income = Column(Integer)
    location = Column(String)
    tax_amount = Column(Integer)

In [54]:
# incomes generated from log normal distribution
incomes = np.random.lognormal(mean=10.5, sigma=.6, size=1000).tolist()
incomes = [100*int(income) for income in incomes]

In [55]:
for _ in range(1000):
    user_year = random.randint(1960, 2000)
    user_income = random.choice(incomes)
    user_gender = random.choice(["male", "female"])

    if user_gender == 'female':
        user_base_income = user_income * 0.9

    new_user = User(
        id=str(uuid.uuid4()),
        name="User Name",
        password="Password",
        gender=user_gender,
        date_of_birth=str(user_year) + "-01-01",
    )

    session.add(new_user)

    location = random.choice(["dhaka", "dhaka", "dhaka", "dhaka", "chittagong", "chittagong", "chittagong", "city", "city", "non_city"])

    for year in range(2015, 2022):
        new_tax_details = TaxDetails(
            tax_id=str(uuid.uuid4()),
            user_id=new_user.id,
            year=year,
            income=user_income,
            taxable_income=get_taxable_income(
                user_income, new_user.gender, year - user_year + 1
            ),
            location=location,
            tax_amount=calculate_final_tax(
                user_income, new_user.gender, year - user_year + 1, location
            ),
        )
        session.add(new_tax_details)

        user_income *= random.uniform(0.9, 1.2)

    session.commit()

# takes 18 min to run